In [2]:
import pandas as pd
from sqlalchemy import create_engine

In [3]:
engine = create_engine("mysql+mysqldb://danielj:@localhost/ecotest")
con = engine.connect()

In [70]:
tables_results = con.execute('show tables')
tables = tables_results.fetchall()

for table in tables:
    description_results = con.execute('DESCRIBE %s' % table[0])
    description_df = pd.DataFrame(description_results.fetchall(),columns=['Field','Type','Null','Key','Default','Extra'])
    print 'Table %s Column info:' % table[0]
    print description_df
    print ''
    

Table labor_data_00_04 Column info:
       Field        Type Null  Key Default Extra
0      index  bigint(20)  YES  MUL    None      
1  series_id        text  YES         None      
2       year  bigint(20)  YES         None      
3     period        text  YES         None      
4      value      double  YES         None      

Table labor_data_05_09 Column info:
       Field        Type Null  Key Default Extra
0      index  bigint(20)  YES  MUL    None      
1  series_id        text  YES         None      
2       year  bigint(20)  YES         None      
3     period        text  YES         None      
4      value        text  YES         None      

Table labor_data_10_14 Column info:
       Field        Type Null  Key Default Extra
0      index  bigint(20)  YES  MUL    None      
1  series_id        text  YES         None      
2       year  bigint(20)  YES         None      
3     period        text  YES         None      
4      value      double  YES         None      

Table l

In [133]:
subquery = 'SELECT ser.*, per.period_name as begin_name '\
           'FROM labor_legend_series AS ser '\
           'INNER JOIN labor_legend_period AS per '\
                   'ON ser.begin_period = per.period'
            
query='SELECT join1.series_id, join1.begin_name '\
                 'FROM (%s) as join1 ' % subquery

df = pd.read_sql(query, con=con.connection)

In [139]:
subquery = 'SELECT ser.*, per.period_name as begin_name '\
           'FROM labor_legend_series AS ser '\
           'INNER JOIN labor_legend_period AS per '\
                   'ON ser.begin_period = per.period'
            
query = 'SELECT join1.*, per.period_name as end_period_name '\
        'FROM (%s) as join1 ' %(subquery) +\
        'INNER JOIN labor_legend_period AS per '\
                'ON join1.end_period = per.period'


df = pd.read_sql(query , con=con.connection)

In [138]:
print df.columns
df.head()

Index([u'index', u'series_id', u'area_type_code', u'area_code',
       u'measure_code', u'seasonal', u'srd_code', u'series_title',
       u'footnote_codes', u'begin_year', u'begin_period', u'end_year',
       u'end_period', u'begin_name', u'end_period_name'],
      dtype='object')


,index,series_id,area_type_code,area_code,measure_code,seasonal,srd_code,series_title,footnote_codes,begin_year,begin_period,end_year,end_period,begin_name,end_period_name
0,0,LASBS060000000000003,L,BS0600000000000,3,S,6,"Unemployment Rate: Balance of California, stat...",None,1976,M01,2015,M07,January,July
1,1,LASBS060000000000004,L,BS0600000000000,4,S,6,"Unemployment: Balance of California, state les...",None,1976,M01,2015,M07,January,July
2,2,LASBS060000000000005,L,BS0600000000000,5,S,6,"Employment: Balance of California, state less ...",None,1976,M01,2015,M07,January,July
3,3,LASBS060000000000006,L,BS0600000000000,6,S,6,"Labor Force: Balance of California, state less...",None,1976,M01,2015,M07,January,July
4,4,LASBS120000000000003,L,BS1200000000000,3,S,12,"Unemployment Rate: Balance of Florida, state l...",None,1990,M01,2015,M07,January,July


In [18]:
name = 'test'
print tuple(name for i in range (5))

('test', 'test', 'test', 'test', 'test')


In [48]:
'''
dat.series_id
dat.year
dat.period
dat.value

ser.series_id
ser.area_type_code
ser.area_code
ser.measure_code
ser.seasonal
ser.srd_code
ser.series_title
ser.begin_year
ser.begin_period
ser.end_year
ser.end_period

per.period
per.period_abbr
per.period_name
     
meas.measure_code
meas.measure_text
'''


#join period variables in 
ser_per_subjoin1 =  'SELECT ser.*, per.period_name as begin_month '\
                   'FROM labor_legend_series AS ser '\
                   'LEFT OUTER JOIN labor_legend_period AS per '\
                           'ON ser.begin_period = per.period'
            
ser_per_subjoin2 =  'SELECT join1.*, per.period_name as end_month '\
                'FROM (%s) as join1 ' %(ser_per_subjoin1) +\
                'LEFT OUTER JOIN labor_legend_period AS per '\
                        'ON join1.end_period = per.period'

ser_per_name = 'ser_per_join'
ser_per_variables = '%s.series_id, %s.area_type_code, %s.area_code, %s.measure_code, %s.seasonal, ' %tuple(ser_per_name for i in range(5))+\
            '%s.srd_code, %s.series_title, %s.begin_year, %s.begin_period, %s.begin_month, %s.end_year, %s.end_period, %s.end_month' %tuple(ser_per_name for i in range(8))



ser_per_join = 'SELECT %s ' %(ser_per_variables) +\
        'FROM (%s) as %s' %(ser_per_subjoin2,ser_per_name)


df = pd.read_sql(ser_per_join , con=con.connection)
print df.columns
print len(df)

Index([u'series_id', u'area_type_code', u'area_code', u'measure_code',
       u'seasonal', u'srd_code', u'series_title', u'begin_year',
       u'begin_period', u'begin_month', u'end_year', u'end_period',
       u'end_month'],
      dtype='object')
33176


In [47]:
serper_meas_subjoin = 'SELECT serper.*, meas.measure_text '\
                       'FROM (%s) AS serper ' %(ser_per_join) +\
                       'LEFT OUTER JOIN labor_legend_measure AS meas '\
                                   'ON serper.measure_code = meas.measure_code'
serper_meas_name = 'serper_meas_join'
serper_meas_variables = '%s.series_id, %s.area_type_code, %s.area_code, %s.measure_text, %s.seasonal, ' %tuple(serper_meas_name for i in range(5))+\
            '%s.srd_code, %s.series_title, %s.begin_year, %s.begin_period, %s.begin_month, %s.end_year, %s.end_period, %s.end_month' %tuple(serper_meas_name for i in range(8))

serper_meas_join = 'SELECT %s ' %(serper_meas_variables) +\
        'FROM (%s) as %s' %(serper_meas_subjoin,serper_meas_name)
        

df = pd.read_sql(serper_meas_join , con=con.connection)
print df.columns
len(df)

Index([u'series_id', u'area_type_code', u'area_code', u'measure_text',
       u'seasonal', u'srd_code', u'series_title', u'begin_year',
       u'begin_period', u'begin_month', u'end_year', u'end_period',
       u'end_month'],
      dtype='object')


33176

In [ ]:
data_join = 'SELECT data.year, data.period, data.value, serpermeas.* '\
            'FROM labor_data_10_14 AS data '\
            'LEFT OUTER JOIN (%s) AS serpermeas ' %(serper_meas_join)+\
                         'ON data.series_id = serpermeas.series_id'
            
df = pd.read_sql(data_join , con=con.connection)
print df.columns
len(df)